To generate text2cypher dataset based on your own data, you could refer to: https://github.com/tomasonjo/text2cypher

# Setup environment
- python==3.9
- torch==2.3.0 (also supported torch 2.1, 2.2: https://github.com/unslothai/unsloth)
- xformers==0.0.26
- trl==0.8.6
- numpy==1.26.4
- bitsandbytes==0.42.0
- accelerate==0.33
- peft==0.12.0
- unslothai==July-Llama-2024 (git+https://github.com/unslothai/unsloth.git)

In [76]:
import os
os.environ['REQUESTS_CA_BUNDLE'] = '/u01/workspace/hoangp46/vt.crt'
proxy = 'http://localhost:3128'
os.environ['http_proxy'] = proxy
os.environ['https_proxy'] = proxy
os.environ['HTTP_PROXY'] = proxy
os.environ['HTTPS_PROXY'] = proxy
os.environ['no_proxy'] = 'localhost'
os.environ['NO_PROXY'] = 'localhost'

In [64]:
# # Installs Unsloth, Xformers (Flash Attention) and all other packages!.
# !python3.9 -m pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# !python3.9 -m pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
# !python3.9 -m pip install langchain_community neo4j langchain

# Fine Tuning using Unsloth

The code below is from the Unsloth repository: https://colab.research.google.com/drive/135ced7oHytdxu3N2DNe1Z0kqjyYIkDXp?usp=sharing

**Load pretrained model**

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    cache_dir='/u01/workspace/hoangp46/cache/hub'
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: NVIDIA A100 80GB PCIe. Max memory: 79.256 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [4]:
# Save model to local path
model_name = 'llama-3-8b-bnb-4bit'
model.save_pretrained(f'/u01/workspace/hoangp46/models/{model_name}')
tokenizer.save_pretrained(f'/u01/workspace/hoangp46/models/{model_name}')

('/u01/workspace/hoangp46/models/llama-3-8b-bnb-4bit/tokenizer_config.json',
 '/u01/workspace/hoangp46/models/llama-3-8b-bnb-4bit/special_tokens_map.json',
 '/u01/workspace/hoangp46/models/llama-3-8b-bnb-4bit/tokenizer.json')

In [65]:
# Try load model from local path
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = f'/u01/workspace/hoangp46/models/{model_name}',
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: NVIDIA A100 80GB PCIe. Max memory: 79.256 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Inference without finetune

In [66]:
# Set model to inference mode
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

**Load graph**

In [67]:
from langchain_community.graphs import Neo4jGraph

NEO4J_URL = "neo4j://localhost:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "fireinthehole"
NEO4J_DATABASE = 'recommendations'

graph = Neo4jGraph(url=NEO4J_URL, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE, sanitize=True, enhanced_schema=True)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"


In [68]:
graph.refresh_schema()
print(graph.schema)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"


Node properties:
- **Movie**
  - `url`: STRING Example: "https://themoviedb.org/movie/862"
  - `runtime`: INTEGER Min: 2, Max: 910
  - `revenue`: INTEGER Min: 1, Max: 2787965087
  - `imdbRating`: FLOAT Min: 1.6, Max: 9.6
  - `released`: STRING Example: "1995-11-22"
  - `countries`: LIST Min Size: 1, Max Size: 16
  - `languages`: LIST Min Size: 1, Max Size: 19
  - `plot`: STRING Example: "A cowboy doll is profoundly threatened and jealous"
  - `imdbVotes`: INTEGER Min: 13, Max: 1626900
  - `imdbId`: STRING Example: "0114709"
  - `year`: INTEGER Min: 1902, Max: 2016
  - `poster`: STRING Example: "https://image.tmdb.org/t/p/w440_and_h660_face/uXDf"
  - `movieId`: STRING Example: "1"
  - `tmdbId`: STRING Example: "862"
  - `title`: STRING Example: "Toy Story"
  - `budget`: INTEGER Min: 1, Max: 380000000
  - `id`: STRING Example: "1"
  - `tagline`: STRING Example: "The adventure takes off!"
- **Genre**
  - `name`: STRING Example: "Adventure"
- **User**
  - `userId`: STRING Example: "1"
  - 

In [71]:
# Prompt
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# Queries
questions = ["Who is the oldest director?",
             "Find all directors who have directed a movie in Spanish language.",
             "Give me 5 movies where a director has also acted?",
             "List all movies with an IMDb rating greater than 5 that have been directed by a director born in China."
             ]

def generate_cypher_query(question):
  inputs = tokenizer(
  [
      prompt.format(
          f"Convert text to cypher query based on this schema: {graph.schema}", # instruction
          question, # input
          "", # output - leave this blank for generation!
      )
  ], return_tensors = "pt").to(model.device)

  outputs = model.generate(**inputs, max_new_tokens=256, use_cache=True, temperature=0.000000001, top_p=0.9, top_k=20, do_sample=True)
  result = tokenizer.batch_decode(outputs)
  cypher_query = result[0].split("### Response:")[1].split("###")[0].strip().replace("<|end_of_text|>", "")
  return cypher_query

for q in questions:
    print("\n",q)
    cypher_query = generate_cypher_query(q)
    print(cypher_query)
    try:
        context = graph.query(cypher_query)
        print(f"\x1b[32m Context: {context}\x1b[0m\n")
    except Exception as e:
        print(f"\x1b[31m Error: {e}\x1b[0m\n")


 Who is the oldest director?
MATCH (d:Director) WHERE d.born < ALL (d2:Director) RETURN d ORDER BY d.born DESC LIMIT 1

 Error: Generated Cypher Statement is not valid
{code: Neo.ClientError.Statement.SyntaxError} {message: Variable `d2` not defined (line 1, column 40 (offset: 39))
"MATCH (d:Director) WHERE d.born < ALL (d2:Director) RETURN d ORDER BY d.born DESC LIMIT 1"
                                        ^}


 Find all directors who have directed a movie in Spanish language.
MATCH (d:Director)-[:DIRECTED]->(m:Movie)<-[:IN_LANGUAGE]-(l:Language) WHERE l.name = "Spanish" RETURN d.name AS name, d.url AS url, d.poster AS poster, d.bio AS bio, d.imdbId AS imdbId, d.tmdbId AS tmdbId, d.born AS born, d.died AS died, d.bornIn AS bornIn, d.name AS name, d.url AS url, d.poster AS poster, d.bio AS bio, d.imdbId AS imdbId, d.tmdbId AS tmdbId, d.born AS born, d.died AS died, d.bornIn AS bornIn, d.name AS name, d.url AS url, d.poster AS poster, d.bio AS bio, d.imdbId AS imdbId, d.tmdbId AS t

Adding LORA

In [72]:
# LoRA method
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

## Create a dataset

In [77]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = f"Convert text to cypher query based on this schema: {graph.schema}"
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(instructions, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

Load data from HuggingFace

In [80]:
# from datasets import load_dataset
# dataset = load_dataset("yahma/alpaca-cleaned", split = "train", cache_dir="/u01/workspace/hoangp46/cache/hub/")
# dataset = dataset.map(formatting_prompts_func, batched = True,)
# dataset

Load our own data

we're going to use: https://github.com/tomasonjo/text2cypher/blob/main/datasets/synthetic_gpt4turbo_demodbs/text2cypher_gpt4turbo.csv

In [81]:
import pandas as pd

df = pd.read_csv('./data/text2cypher_gpt4turbo.csv')
df = df[(df['database'] == 'recommendations') & (df['syntax_error'] == False) & (df['timeout'] == False)]
df

,question,cypher,type,database,syntax_error,timeout,returns_results,false_schema
7275,What are the top 5 movies with a runtime great...,MATCH (m:Movie)\nWHERE m.runtime > 120\nRETURN...,Simple Retrieval Queries,recommendations,False,False,True,NaN
7276,List the first 3 genres with movies having an ...,MATCH (m:Movie)-[:IN_GENRE]->(g:Genre)\nWHERE ...,Verbose query,recommendations,False,False,True,NaN
7277,List the first 5 directors who have a biograph...,MATCH (d:Director)\nWHERE d.bio IS NOT NULL\nR...,Simple Retrieval Queries,recommendations,False,False,True,NaN
7278,Which 3 movies have the most detailed plot des...,"MATCH (m:Movie)\nRETURN m.title, m.plot\nORDER...",Simple Retrieval Queries,recommendations,False,False,True,NaN
7279,Show the top 5 actors who have acted in movies...,MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)<-[:DIRE...,Simple Retrieval Queries,recommendations,False,False,True,NaN
...,...,...,...,...,...,...,...,...
8067,Which movies have been acted in by more than 1...,MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)\nWITH m...,Complex Retrieval Queries,recommendations,False,False,True,NaN
8068,Find all movies where the director has directe...,MATCH (d:Director)-[:DIRECTED]->(m:Movie)\nWIT...,Complex Retrieval Queries,recommendations,False,False,False,NaN
8069,Find all movies that have a plot mentioning 'h...,MATCH (m:Movie)\nWHERE m.plot CONTAINS 'hero'\...,Complex Retrieval Queries,recommendations,False,False,True,NaN
8070,Which movies have been rated the highest by us...,"MATCH (u:User)-[r:RATED]->(m:Movie)\nWITH u, c...",Complex Retrieval Queries,recommendations,False,False,True,NaN


In [82]:
df = df[['question','cypher']]
df.rename(columns={'question': 'input','cypher':'output'}, inplace=True)
df.reset_index(drop=True, inplace=True)
df

/tmp/ipykernel_390357/2603659181.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'question': 'input','cypher':'output'}, inplace=True)


,input,output
0,What are the top 5 movies with a runtime great...,MATCH (m:Movie)\nWHERE m.runtime > 120\nRETURN...
1,List the first 3 genres with movies having an ...,MATCH (m:Movie)-[:IN_GENRE]->(g:Genre)\nWHERE ...
2,List the first 5 directors who have a biograph...,MATCH (d:Director)\nWHERE d.bio IS NOT NULL\nR...
3,Which 3 movies have the most detailed plot des...,"MATCH (m:Movie)\nRETURN m.title, m.plot\nORDER..."
4,Show the top 5 actors who have acted in movies...,MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)<-[:DIRE...
...,...,...
757,Which movies have been acted in by more than 1...,MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)\nWITH m...
758,Find all movies where the director has directe...,MATCH (d:Director)-[:DIRECTED]->(m:Movie)\nWIT...
759,Find all movies that have a plot mentioning 'h...,MATCH (m:Movie)\nWHERE m.plot CONTAINS 'hero'\...
760,Which movies have been rated the highest by us...,"MATCH (u:User)-[r:RATED]->(m:Movie)\nWITH u, c..."


In [83]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)
dataset = dataset.map(formatting_prompts_func, batched = True)
dataset

Map:   0%|          | 0/762 [00:00<?, ? examples/s]

Dataset({
    features: ['input', 'output', 'text'],
    num_rows: 762
})

In [84]:
dataset[0]

{'input': 'What are the top 5 movies with a runtime greater than 120 minutes?',
 'output': 'MATCH (m:Movie)\nWHERE m.runtime > 120\nRETURN m\nORDER BY m.runtime DESC\nLIMIT 5',
 'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nConvert text to cypher query based on this schema: Node properties:\n- **Movie**\n  - `url`: STRING Example: "https://themoviedb.org/movie/862"\n  - `runtime`: INTEGER Min: 2, Max: 910\n  - `revenue`: INTEGER Min: 1, Max: 2787965087\n  - `imdbRating`: FLOAT Min: 1.6, Max: 9.6\n  - `released`: STRING Example: "1995-11-22"\n  - `countries`: LIST Min Size: 1, Max Size: 16\n  - `languages`: LIST Min Size: 1, Max Size: 19\n  - `plot`: STRING Example: "A cowboy doll is profoundly threatened and jealous"\n  - `imdbVotes`: INTEGER Min: 13, Max: 1626900\n  - `imdbId`: STRING Example: "0114709"\n  - `year`: INTEGER Min: 1902, Max: 2016\

# Train the model

In [85]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # max_steps = 60,
        num_train_epochs=1,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/762 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [86]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100 80GB PCIe. Max memory = 79.256 GB.
17.826 GB of memory reserved.


NOTE:
- Create symlink libcuda by the command: `cd /lib/x86_64-linux-gnu && sudo ln -s libcuda.so.1 libcuda.so`
- Install python3.9-dev by the command: `sudo apt-get install python3.9-dev`

In [87]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 762 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 95
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.148400
2,1.142700
3,1.146400
4,1.117400
5,1.044300
6,0.932300
7,0.817200
8,0.719900
9,0.609800
10,0.494000


In [90]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

325.2573 seconds used for training.
5.42 minutes used for training.
Peak reserved memory = 17.826 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 22.492 %.
Peak reserved memory for training % of max memory = 0.0 %.


# Inference

In [91]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

inputs = tokenizer(
[
    prompt.format(
        f"Convert text to cypher query based on this schema: {graph.schema}", # instruction
        "Who is the oldest director?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Convert text to cypher query based on this schema: Node properties:
- **Movie**
  - `url`: STRING Example: "https://themoviedb.org/movie/862"
  - `runtime`: INTEGER Min: 2, Max: 910
  - `revenue`: INTEGER Min: 1, Max: 2787965087
  - `imdbRating`: FLOAT Min: 1.6, Max: 9.6
  - `released`: STRING Example: "1995-11-22"
  - `countries`: LIST Min Size: 1, Max Size: 16
  - `languages`: LIST Min Size: 1, Max Size: 19
  - `plot`: STRING Example: "A cowboy doll is profoundly threatened and jealous"
  - `imdbVotes`: INTEGER Min: 13, Max: 1626900
  - `imdbId`: STRING Example: "0114709"
  - `year`: INTEGER Min: 1902, Max: 2016
  - `poster`: STRING Example: "https://image.tmdb.org/t/p/w440_and_h660_face/uXDf"
  - `movieId`: STRING Example: "1"
  - `tmdbId`: STRING Example: "862"
  - `title`: STRING Example: "T

# Save the Finetuned

Local Saving

In [92]:
local_path = "/u01/workspace/hoangp46/ft_models/llama3-text2cypher"
model.save_pretrained(local_path) # Local saving
tokenizer.save_pretrained(local_path)

('/u01/workspace/hoangp46/ft_models/llama3-text2cypher/tokenizer_config.json',
 '/u01/workspace/hoangp46/ft_models/llama3-text2cypher/special_tokens_map.json',
 '/u01/workspace/hoangp46/ft_models/llama3-text2cypher/tokenizer.json')

Online Saving to HuggingFace

In [93]:
# # should have write access
# model.push_to_hub("hoangph3/llama3-text2cypher", token=hf_api)
# tokenizer.push_to_hub("hoangph3/llama3-text2cyphers", token=hf_api)

# Load Finetuned Model from HuggingFace

In [94]:
from unsloth import FastLanguageModel

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=local_path, # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: NVIDIA A100 80GB PCIe. Max memory: 79.256 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [95]:
inputs = tokenizer(
[
    prompt.format(
        f"Convert text to cypher query based on this schema: {graph.schema}", # instruction
        "Who is the oldest director?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
result = tokenizer.batch_decode(outputs)
response = result[0].split("### Response:")[1].split("###")[0].strip().replace("<|end_of_text|>", "")
print(response)

MATCH (d:Director)
WHERE d.born IS NOT NULL
RETURN d.name, d.born, d.died
ORDER BY d.born ASC
LIMIT 1


# Evaluating

In [127]:
# Prompt
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# Queries
questions = ["Who is the oldest director?",
             "Find all directors who have directed a movie in Spanish language.",
             "Give me 5 movies where a director has also acted?",
             "List all movies with an IMDb rating greater than 5 that have been directed by a director born in China."
             ]

def generate_cypher_query(question):
  inputs = tokenizer(
  [
      prompt.format(
          f"Convert text to cypher query based on this schema: {graph.schema}", # instruction
          question, # input
          "", # output - leave this blank for generation!
      )
  ], return_tensors = "pt").to(model.device)

  outputs = model.generate(**inputs, max_new_tokens=256, use_cache=True, temperature=0.000000001, top_p=0.9, top_k=10, do_sample=True)
  result = tokenizer.batch_decode(outputs)
  cypher_query = result[0].split("### Response:")[1].split("###")[0].strip().replace("<|end_of_text|>", "")
  return cypher_query

for q in questions:
    print("\n",q)
    cypher_query = generate_cypher_query(q)
    print(cypher_query)
    try:
        context = graph.query(cypher_query)
        print(f"\x1b[32m Context: {context}\x1b[0m\n")
    except Exception as e:
        print(f"\x1b[31m Error: {e}\x1b[0m\n")


 Who is the oldest director?
MATCH (d:Director)
WHERE d.born IS NOT NULL
RETURN d.name, d.born, d.died
ORDER BY d.born ASC
LIMIT 1
 Context: [{'d.name': 'Georges Méliès', 'd.born': neo4j.time.Date(1861, 12, 8), 'd.died': neo4j.time.Date(1938, 1, 21)}]


 Find all directors who have directed a movie in Spanish language.
MATCH (d:Director)-[:DIRECTED]->(m:Movie)
WHERE 'Spanish' IN m.languages
RETURN d.name, collect(m.title) AS moviesDirected
 Context: [{'d.name': 'Alejandro Jodorowsky', 'moviesDirected': ['Topo, El', 'Fando and Lis (Fando y Lis)']}, {'d.name': 'Alfonso Arau', 'moviesDirected': ['Like Water for Chocolate (Como agua para chocolate)']}, {'d.name': 'Abel Ferrara', 'moviesDirected': ['King of New York']}, {'d.name': 'Nacho Vigalondo', 'moviesDirected': ['Timecrimes (Cronocrímenes, Los)']}, {'d.name': 'Luis Buñuel', 'moviesDirected': ['Tristana', 'Nazarin (Nazarín)', 'Simon of the Desert (Simón del desierto)', 'Viridiana', 'Exterminating Angel, The (Ángel exterminador, El)']}

Unsloth has not integrated in Langchain, so need little adjustment

In [141]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks.tracers import ConsoleCallbackHandler

# LLM local
# llm = ChatOpenAI(
#     openai_api_base="http://localhost:8001/v1",
#     model_name="meta-llama3-8b",
#     temperature=0.0,
#     openai_api_key="secret",
#     streaming=True,
#     max_tokens=512
# )
llm = ChatOpenAI(temperature=0.0, model_name='gpt-3.5-turbo')

CYPHER_QA_TEMPLATE = """You convert context to a final answer. Understand the question, the context, then generate result.
Here is an example:

Question: Who is the director of Harry Potter 1 and 8?
Context: [{{d.name: Chris Columbus, d.born: 10 September 1958}},{{d.name: David Yates, d.born: 8 October 1963}}]
Helpful Answer: Chris Columbus and David Yates is the director of Harry Potter

Follow this example when generating answers.
Answer in short, don't hallucinate!
Question: {question}
Information: {context}
Helpful Answer:
"""

qa_prompt = ChatPromptTemplate.from_template(CYPHER_QA_TEMPLATE)
output_parser = StrOutputParser()
chain = qa_prompt | llm | output_parser

In [142]:
# Queries
questions = ["Who is the oldest director?",
             "Find all directors who have directed a movie in Spanish language.",
             "Give me 5 movies where a director has also acted?",
             "List all movies with an IMDb rating greater than 5 that have been directed by a director born in China."
             ]

def generate_cypher_query(question):
  inputs = tokenizer(
  [
      prompt.format(
          f"Convert text to cypher query based on this schema: {graph.schema}", # instruction
          question, # input
          "", # output - leave this blank for generation!
      )
  ], return_tensors = "pt").to(model.device)

  outputs = model.generate(**inputs, max_new_tokens=256, use_cache=True, temperature=0.000000001, top_p=0.9, top_k=10, do_sample=True)
  result = tokenizer.batch_decode(outputs)
  cypher_query = result[0].split("### Response:")[1].split("###")[0].strip().replace("<|end_of_text|>", "")
  return cypher_query

for q in questions:
    print("\n",q)
    cypher_query = generate_cypher_query(q)
    print(cypher_query)
    context = graph.query(cypher_query)
    print('context: ', context)
    # result = chain.invoke({"context":context , "question":q}, config={'callbacks': [ConsoleCallbackHandler()]})
    result = chain.invoke({"context":context , "question":q})
    print(f"\x1b[32mResponse: {result}\x1b[0m\n")


 Who is the oldest director?
MATCH (d:Director)
WHERE d.born IS NOT NULL
RETURN d.name, d.born, d.died
ORDER BY d.born ASC
LIMIT 1
context:  [{'d.name': 'Georges Méliès', 'd.born': neo4j.time.Date(1861, 12, 8), 'd.died': neo4j.time.Date(1938, 1, 21)}]
Response: Georges Méliès is the oldest director.


 Find all directors who have directed a movie in Spanish language.
MATCH (d:Director)-[:DIRECTED]->(m:Movie)
WHERE 'Spanish' IN m.languages
RETURN d.name, collect(m.title) AS moviesDirected
context:  [{'d.name': 'Alejandro Jodorowsky', 'moviesDirected': ['Topo, El', 'Fando and Lis (Fando y Lis)']}, {'d.name': 'Alfonso Arau', 'moviesDirected': ['Like Water for Chocolate (Como agua para chocolate)']}, {'d.name': 'Abel Ferrara', 'moviesDirected': ['King of New York']}, {'d.name': 'Nacho Vigalondo', 'moviesDirected': ['Timecrimes (Cronocrímenes, Los)']}, {'d.name': 'Luis Buñuel', 'moviesDirected': ['Tristana', 'Nazarin (Nazarín)', 'Simon of the Desert (Simón del desierto)', 'Viridiana', 'Ext